### RAG와 LLM을 활용한 질의응답 실습
- LLM API 호출 시 RAG를 통해 연관 정보 추출 후 Prompt에 추가하는 실습
- RAG 추가 전후로 평가 진행

In [1]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [2]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

### 평가
- RAGAS의 Answer Correctedness 평가 지표
  - 모델의 답변과 실제 정답 간의 일치율 측정

### Baseline (w/o RAG)

In [3]:
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [01:21<00:00,  8.18s/it]


In [4]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import answer_correctness


data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

/Users/rein20/Documents/lectures/ai-service/retrieval-augmented-generation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Evaluating: 100%|██████████| 10/10 [01:17<00:00,  7.73s/it]


{'answer_correctness': 0.4985}

In [5]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [6]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 선진국을 중심으로 경제 성장률이 둔화되면서 중앙은행들이 경기 부양을 위해 금리를 낮추는 정책을 채택하게 되었습니다. 이는 저금리 현상의 주요 원인 중 하나입니다.

2. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 소비와 투자가 줄어들고, 저축이 증가하는 경향이 있습니다. 이는 금리 하락을 유도하는 요인으로 작용합니다.

3. **글로벌 금융 위기**: 2008년 글로벌 금융 위기 이후, 각국 중앙은행들은 경제 회복을 위해 금리를 낮추고 양적 완화 정책을 시행했습니다. 이러한 정책들은 장기적으로 저금리 환경을 조성하는 데 기여했습니다.

4. **디플레이션 압력**: 일부 국가에서는 디플레이션(물가 하락) 압력이 존재하여 중앙은행들이 금리를 낮추는 정책을 통해 물가 상승을 유도하려고 합니다.

5. **국제 자본 이동**: 글로벌화로 인해 자본이 국경을 넘어 자유롭게 이동하면서, 금리가 낮은 국가로 자본이 유입되는 현상이 발생합니다. 이는 금리 하락을 더욱 촉진시킵니다.

6. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 억제하는 요인으로 작용합니다. 중앙은행들은 이러한 상황에서 금리를 낮추는 경향이 있습니다.

7. **정책적 요인**: 각국 정부와 중앙은행들이 경제 안정과 성장을 위해 저금리 정책을 지속적으로 유지하는 경우도 있습니다. 이는 경기 부양과 고용 창출을 목표로 하는 정책적 결정입니다.

이와 같은 여러 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되고 있습니다.


### LLM + RAG
- text-embedding-3-large 임베딩 모델 사용

In [7]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity


def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:55<00:00,  5.60s/it]


In [8]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_relevancy, context_recall, context_precision


data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

Evaluating: 100%|██████████| 10/10 [01:00<00:00,  6.05s/it]


{'answer_correctness': 0.6225}

In [9]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행하였습니다. 이러한 정책은 금리를 낮추고 유동성을 공급하여 경제를 부양하려는 목적이 있었습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여하였습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상(global savings glut)이 발생하였습니다.

3. **코로나 팬데믹**: 최근 코로나 사태로 인해 경제가 큰 타격을 받으면서, 각국 중앙은행들은 다시 한 번 완화적 통화정책을 강화하였습니다. 이로 인해 저금리 기조가 더욱 부각되었습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 지속되고 있습니다.


In [10]:
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
